In [16]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
# 禁用所有警告
warnings.filterwarnings("ignore")

In [17]:
df = pd.read_excel('EDI-Family.xlsx',index_col = 0)
C_df = pd.read_excel("data\Concentration in Biota.xlsx")
P_df =pd.read_excel("data\Production.xlsx",index_col=0).fillna(0) # 无对应鱼种的捕捞数据，视为0
print(P_df.shape)
P_df.head()
# P_df.isna().sum()
code_df =  pd.read_excel("data\Meaning of Code.xlsx")
W_c_df =pd.read_excel('Country-Family-Weight.xlsx',index_col=0,header=[0,1])
W_c_E_code_df = pd.read_excel('Export-Weight.xlsx',index_col=0,header=[0,1])
pop_df = pd.read_excel("data\Population.xlsx",index_col=0,names=['pop'])
code_family = pd.read_excel("data\Code-Family.xlsx")
code_family.head()

(33, 212)


,Anchovies,Cod-likes,Flatfishes,Herring-likes,Perch-likes,"Salmon, smelts, etc",Scorpionfishes,Sharks & rays,Tuna & billfishes,Other fishes & inverts
0,30242.0,30250.0,30221.0,30240.0,30244.0,30212.0,NaN,30265.0,30194,NaN
1,NaN,30251.0,30222.0,30241.0,30245.0,30213.0,NaN,30281.0,30195,NaN
2,NaN,30252.0,30223.0,30243.0,30246.0,30214.0,NaN,30282.0,30231,NaN
3,NaN,30253.0,30224.0,30261.0,30264.0,30219.0,NaN,30375.0,30232,NaN
4,NaN,30254.0,30229.0,30350.0,30284.0,30310.0,NaN,30381.0,30233,NaN


In [18]:
# 变量初始化
chem_df_lis = []

code_lis = code_df.columns
print(code_lis)
C_fami = ['mean','p025','p975']
chem_lis = np.unique(C_df.Chemical)

pop_df = pd.read_excel("data\Population.xlsx",index_col=0,names=['pop'])
#country_lis = np.unique(C_df.Country.to_list())
country_lis = pop_df.index.to_list()
family_lis = np.unique(W_c_df.columns.get_level_values(0).to_list())
print(chem_lis)
print(chem_lis.size)
print(len(country_lis))

Index([30194, 30195, 30212, 30213, 30214, 30219, 30221, 30222, 30223, 30224,
       ...
       30481, 30483, 30484, 30486, 30487, 30488, 30491, 30495, 30496, 30497],
      dtype='int64', length=111)
['PFBS' 'PFDA' 'PFHpA' 'PFHxA' 'PFHxS' 'PFNA' 'PFOA' 'PFOS' 'PFUnDA']
9
33


In [19]:
def init_data():
    dic ={}
    for _c in country_lis:
        dic[_c] =[]
    return dic

In [20]:
from tqdm import tqdm
for _chem in chem_lis:
    W_c_df =pd.read_excel('Country-Family-Weight.xlsx',
                        index_col=0,header=[0,1],sheet_name=_chem)
    W_c_E_code_df = pd.read_excel('Export-Weight.xlsx',
                                index_col=0,header=[0,1],sheet_name=_chem)

    excel_writer = pd.ExcelWriter(f'EDI-corresponding-to-the-trade_{_chem}.xlsx', engine='xlsxwriter')

    pop_df = pd.read_excel("data\Population.xlsx",index_col=0,names=['pop'])
    dic_mean = init_data()
    dic_p025 = init_data()
    dic_p975 = init_data()
    for _c in tqdm(country_lis):
        _pop = pop_df.loc[_c,'pop']
        W_c_A_Import_df = pd.read_excel(f'Import-Weight-{_chem}.xlsx',
                                    index_col=0,header=[0,1],sheet_name=_c)
        
        for _c_2 in country_lis:
            if _c == _c_2:
                _mean = 0
                _p025 = 0
                _p975 = 0
                for i in range(len(family_lis)):
                    _mean += W_c_df.loc[_c,(family_lis[i],C_fami[0])]
                    _p025 += W_c_df.loc[_c,(family_lis[i],C_fami[1])]
                    _p975 += W_c_df.loc[_c,(family_lis[i],C_fami[2])]  
                for i in range(len(code_lis)):
                    _mean -= W_c_E_code_df.loc[_c,(code_lis[i],C_fami[0])]
                    _p025 -= W_c_E_code_df.loc[_c,(code_lis[i],C_fami[1])]
                    _p975 -= W_c_E_code_df.loc[_c,(code_lis[i],C_fami[2])]
                dic_mean[_c].append(_mean/(365*65*_pop))
                dic_p025[_c].append(_p025/(365*65*_pop))
                dic_p975[_c].append(_p975/(365*65*_pop))
            else:
                _mean = 0
                _p025 = 0
                _p975 = 0
                for i in range(len(code_lis)):
                    #if _c == 'Canada' and _c_2 == 'United States of America' and W_c_A_Import_df.loc[_c_2,(code_lis[i],C_fami[0])] != 0 :
                    #    print(f'mean{_mean} + {W_c_A_Import_df.loc[_c_2,(code_lis[i],C_fami[0])]} = {W_c_A_Import_df.loc[_c_2,(code_lis[i],C_fami[0])]+_mean} ')
                    _mean += W_c_A_Import_df.loc[_c_2,(code_lis[i],C_fami[0])]
                    _p025 += W_c_A_Import_df.loc[_c_2,(code_lis[i],C_fami[1])]
                    _p975 += W_c_A_Import_df.loc[_c_2,(code_lis[i],C_fami[2])]
                dic_mean[_c].append(_mean/(365*65*_pop))
                dic_p025[_c].append(_p025/(365*65*_pop))
                dic_p975[_c].append(_p975/(365*65*_pop))
                #if _c == 'Canada' and _c_2 == 'United States of America' :
                #        print(f'{_mean/(365*65*_pop)}')
    df_mean = pd.DataFrame(dic_mean,columns=country_lis,index=country_lis).T
    df_p025 = pd.DataFrame(dic_p025,columns=country_lis,index=country_lis).T
    df_p975 = pd.DataFrame(dic_p975,columns=country_lis,index=country_lis).T

    df_mean.to_excel(excel_writer,sheet_name='MEAN',index=True)
    df_p025.to_excel(excel_writer,sheet_name='p025',index=True)
    df_p975.to_excel(excel_writer,sheet_name='p975',index=True)

    excel_writer.close()

100%|██████████| 33/33 [00:39<00:00,  1.21s/it]


In [21]:
###############################
###############################

In [22]:
excel_writer = pd.ExcelWriter(f'EDI-Family-NO Trade.xlsx', engine='xlsxwriter')

for _chem in chem_lis:
    W_c_f_df = pd.read_excel("Country-Family-Weight.xlsx",index_col=0,header=[0,1],sheet_name=_chem)
    pop_df = pd.read_excel("data\\Population.xlsx",index_col=0,names=['pop'])
    _df = W_c_f_df.copy()

    for _family in np.unique(W_c_f_df.columns.get_level_values(0).tolist()):
        for c_type in C_fami:
            _lis =[]
            W_c_arr = W_c_f_df[(_family,c_type)]
            
            for i in range(len(W_c_arr)):
                _pop = pop_df.loc[W_c_f_df.index[i],'pop']
                _lis.append(W_c_arr[i]/(365*65*_pop))
            _df[(_family,c_type)] = _lis
    _df.to_excel(excel_writer,sheet_name=_chem,index=True)
excel_writer.close()

In [23]:
####################################

In [24]:
##################################

In [25]:
###############################

In [26]:
excel_writer = pd.ExcelWriter('EDI-Family-Negative-Process.xlsx', engine='xlsxwriter')
for _chem in chem_lis: 
    code_family = pd.read_excel("data\Code-Family.xlsx")
    W_c_df =pd.read_excel('Country-Family-Weight.xlsx',index_col=0,header=[0,1],
                        sheet_name=_chem)
    W_c_E_code_df = pd.read_excel('Export-Weight.xlsx',index_col=0,header=[0,1],
                                sheet_name=_chem)
    pop_df = pd.read_excel("data\Population.xlsx",index_col=0,names=['pop'])

    _df = W_c_df.copy()
    for _family in tqdm(np.unique(W_c_df.columns.get_level_values(0).to_list())):
        code_in_family = code_family[_family]
        for _c_type in C_fami:
            _W_c_arr = W_c_df[(_family,_c_type)]
            _lis = []
            
            for i in range(len(_df.index)):
                _W_c = _W_c_arr[i]
                
                #if _family =='Salmonidae 鲑科' and _df.index[i] =='Canada' and _chem =='PFOA':
                #    print(f'W_C_{_c_type} = {_W_c}')
                    
                for _code in code_in_family:
                    if np.isnan(_code):
                        continue
                    #_code = '0'+str(int(_code))
                    _we = W_c_E_code_df.loc[(_df.index[i],(_code,_c_type))]
                    _W_c -= _we
                    #if _family =='Salmonidae 鲑科' and _df.index[i] =='Canada' \
                    #    and _chem =='PFOS' and _we!=0 and _c_type == C_fami[2]:
                    #    print(_code)
                    #    print(f'W_C_{_c_type}- {_we}  ={_W_c}')
                if _W_c < 0 :
                    _W_c = 0
                W_c_A_Import_df = pd.read_excel(f'Import-Weight-{_chem}.xlsx',
                        index_col=0,header=[0,1],
                        sheet_name=_df.index[i])
                
                for _code in code_in_family:
                    if np.isnan(_code):
                        continue
                    for _x in W_c_A_Import_df[(_code,_c_type)]:
                        _W_c+=_x
                        
                _pop= pop_df.loc[_df.index[i],'pop']
                Weight = 65 # 成年人体重kg                    
                _lis.append((_W_c)/(365*_pop*Weight))
                
            _df[(_family,_c_type)] = _lis
    
    _df.to_excel(excel_writer,sheet_name=_chem,index=True)
excel_writer.close()

100%|██████████| 10/10 [05:19<00:00, 31.94s/it]


In [31]:
code_family.columns.__len__()

10

In [28]:
###########
###########
###########

In [29]:
from tqdm import tqdm
code_family = pd.read_excel("data\Code-Family.xlsx")
def relu(a):
    if a <0 :
        return 0
    else:
        return a
for _chem in chem_lis:
    W_c_df =pd.read_excel('Country-Family-Weight.xlsx',
                        index_col=0,header=[0,1],sheet_name=_chem)
    W_c_E_code_df = pd.read_excel('Export-Weight.xlsx',
                                index_col=0,header=[0,1],sheet_name=_chem)

    excel_writer = pd.ExcelWriter(f'EDI-corresponding-to-the-trade_{_chem}_Negative-Process.xlsx', engine='xlsxwriter')
    dic_mean = init_data()
    dic_p025 = init_data()
    dic_p975 = init_data()
    for _c in tqdm(country_lis):
        _pop = pop_df.loc[_c,'pop']
        W_c_A_Import_df = pd.read_excel(f'Import-Weight-{_chem}.xlsx',
                                    index_col=0,header=[0,1],sheet_name=_c)
        
        for _c_2 in country_lis:
            if _c == _c_2:
                _mean = 0
                _p025 = 0
                _p975 = 0
                for i in range(len(family_lis)):
                    _mean_ = 0
                    _p025_ = 0
                    _p975_ = 0
                    code_in_family = code_family[family_lis[i]]
                    _mean_ += W_c_df.loc[_c,(family_lis[i],C_fami[0])]
                    _p025_ += W_c_df.loc[_c,(family_lis[i],C_fami[1])]
                    _p975_ += W_c_df.loc[_c,(family_lis[i],C_fami[2])]  
                    
                    for _code in code_in_family:
                        if np.isnan(_code):
                            continue
                        #_code = '0'+str(int(_code))
                        _mean_ -= W_c_E_code_df.loc[(_c,(_code,C_fami[0]))]
                        _p025_ -= W_c_E_code_df.loc[(_c,(_code,C_fami[1]))]
                        _p975_ -= W_c_E_code_df.loc[(_c,(_code,C_fami[2]))]
                    
                    _mean += relu(_mean_)    
                    _p025 += relu(_p025_)    
                    _p975 += relu(_p975_)    
                dic_mean[_c].append(_mean/(365*65*_pop))
                dic_p025[_c].append(_p025/(365*65*_pop))
                dic_p975[_c].append(_p975/(365*65*_pop))
            else:
                _mean = 0
                _p025 = 0
                _p975 = 0
                for i in range(len(code_lis)):
                    _mean += W_c_A_Import_df.loc[_c_2,(code_lis[i],C_fami[0])]
                    _p025 += W_c_A_Import_df.loc[_c_2,(code_lis[i],C_fami[1])]
                    _p975 += W_c_A_Import_df.loc[_c_2,(code_lis[i],C_fami[2])]
                dic_mean[_c].append(_mean/(365*65*_pop))
                dic_p025[_c].append(_p025/(365*65*_pop))
                dic_p975[_c].append(_p975/(365*65*_pop))

    df_mean = pd.DataFrame(dic_mean,columns=country_lis,index=country_lis).T
    df_p025 = pd.DataFrame(dic_p025,columns=country_lis,index=country_lis).T
    df_p975 = pd.DataFrame(dic_p975,columns=country_lis,index=country_lis).T

    df_mean.to_excel(excel_writer,sheet_name='MEAN',index=True)
    df_p025.to_excel(excel_writer,sheet_name='p025',index=True)
    df_p975.to_excel(excel_writer,sheet_name='p975',index=True)

    excel_writer.close()

100%|██████████| 33/33 [00:36<00:00,  1.11s/it]


In [30]:
chem_lis

array(['PFBS', 'PFDA', 'PFHpA', 'PFHxA', 'PFHxS', 'PFNA', 'PFOA', 'PFOS',
       'PFUnDA'], dtype=object)